In [124]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
import statsmodels as sm

from pyjstat import pyjstat
from ecbdata import ecbdata
import requests

from bs4 import BeautifulSoup
import re

import os
import warnings
warnings.filterwarnings("ignore")

from obiwan import *

In [125]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [149]:
filename = 'GVM_247_2024_IRB_PD_PE_Backtesting à Calibração - Teste score incidentes.xlsx'
file_path = 'C:/Users/B28658/Novo Banco S.A/GVM Team - General/04_IRB/PROCESSO_VALIDACAO_2024/05_PD_PE/01_ANÁLISES/' + filename

In [150]:
df = pd.read_excel(file_path, sheet_name='LongRun_Data_Total (P)')

In [151]:
df.head()

,Unnamed: 0,BONS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unrated,Unnamed: 12,Unnamed: 13,Unnamed: 14,DEFAULTS,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unrated.1,Unnamed: 26,Unnamed: 27,Unnamed: 28,DRs,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unrated.2,Unnamed: 40,Unnamed: 41
0,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated > 500,TOTAL,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated>500,TOTAL,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated >500,TOTAL
1,NaN,data,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,NaN,NaN
2,2010.0,201001,1303,1621,1451,1911,2086,2182,2768,2441,2033,5013,16493,22809,NaN,201001.0,87,4,7,19,37,50,84,137,420,339,758,1184,NaN,201001.0,0.066769,0.002468,0.004824,0.009942,0.017737,0.022915,0.030347,0.056125,0.206591,0.067624,0.045959,0.051909
3,2010.0,201002,1369,1687,1503,1963,2161,2197,2848,2512,2045,4584,16916,22869,NaN,201002.0,100,3,8,20,32,56,82,130,440,334,771,1205,NaN,201002.0,0.073046,0.001778,0.005323,0.010188,0.014808,0.025489,0.028792,0.051752,0.215159,0.072862,0.045578,0.052691
4,2010.0,201003,1370,1747,1577,2010,2212,2283,2946,2583,2151,4135,17509,23014,NaN,201003.0,98,2,10,16,33,57,87,128,457,289,790,1177,NaN,201003.0,0.071533,0.001145,0.006341,0.00796,0.014919,0.024967,0.029532,0.049555,0.212459,0.069891,0.04512,0.051143


In [152]:
df.columns

Index(['Unnamed: 0', 'BONS', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unrated', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'DEFAULTS', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unrated.1', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'DRs', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unrated.2', 'Unnamed: 40', 'Unnamed: 41'],
      dtype='object')

In [153]:
df_bons = df[['Unnamed: 0', 'BONS', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unrated', 'Unnamed: 12', 'Unnamed: 13']]#.set_index('Unnamed: 0')

df_bons.head()

,Unnamed: 0,BONS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unrated,Unnamed: 12,Unnamed: 13
0,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated > 500,TOTAL
1,NaN,data,N,N,N,N,N,N,N,N,N,N,NaN,NaN
2,2010.0,201001,1303,1621,1451,1911,2086,2182,2768,2441,2033,5013,16493,22809
3,2010.0,201002,1369,1687,1503,1963,2161,2197,2848,2512,2045,4584,16916,22869
4,2010.0,201003,1370,1747,1577,2010,2212,2283,2946,2583,2151,4135,17509,23014


In [154]:
df_bons.columns = df_bons.loc[0,:'Unnamed: 13']
df_bons.head()

,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated > 500,TOTAL
0,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated > 500,TOTAL
1,NaN,data,N,N,N,N,N,N,N,N,N,N,NaN,NaN
2,2010.0,201001,1303,1621,1451,1911,2086,2182,2768,2441,2033,5013,16493,22809
3,2010.0,201002,1369,1687,1503,1963,2161,2197,2848,2512,2045,4584,16916,22869
4,2010.0,201003,1370,1747,1577,2010,2212,2283,2946,2583,2151,4135,17509,23014


In [155]:
df_bons = df_bons.drop([0,1])
df_bons.head()

,NaN,NaN,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated > 500,TOTAL
2,2010.0,201001,1303,1621,1451,1911,2086,2182,2768,2441,2033,5013,16493,22809
3,2010.0,201002,1369,1687,1503,1963,2161,2197,2848,2512,2045,4584,16916,22869
4,2010.0,201003,1370,1747,1577,2010,2212,2283,2946,2583,2151,4135,17509,23014
5,2010.0,201004,1441,1946,1757,2128,2285,2249,2853,2422,2091,3889,17731,23061
6,2010.0,201005,1428,1978,1745,2207,2347,2276,2917,2416,2009,3673,17895,22996


In [156]:
df_bons.columns = ['year', 'data' , '<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated > 500',          'TOTAL']

In [157]:
df_bons[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated > 500',          'TOTAL']] = df_bons[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated > 500',          'TOTAL']].astype(float)

In [158]:
df_bons.columns.name = 'BONS'

In [159]:
df_bons = df_bons.drop(columns='year')

In [160]:
df.columns

Index(['Unnamed: 0', 'BONS', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unrated', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'DEFAULTS', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unrated.1', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'DRs', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unrated.2', 'Unnamed: 40', 'Unnamed: 41'],
      dtype='object')

In [161]:
df_defaults = df[['DEFAULTS', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unrated.1', 'Unnamed: 26', 'Unnamed: 27']]

df_defaults.columns = df_defaults.loc[0,'DEFAULTS':'Unnamed: 27']
df_defaults = df_defaults.drop([0,1])
df_defaults = df_defaults.rename(columns={df_defaults.columns[0]: 'data'})

df_defaults[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated>500',          'TOTAL']] = df_defaults[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated>500',          'TOTAL']].astype(float)

df_defaults.columns.name = 'DEFAULTS'

df_defaults.head()

DEFAULTS,data,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated>500,TOTAL
2,201001.0,87.0,4.0,7.0,19.0,37.0,50.0,84.0,137.0,420.0,339.0,758.0,1184.0
3,201002.0,100.0,3.0,8.0,20.0,32.0,56.0,82.0,130.0,440.0,334.0,771.0,1205.0
4,201003.0,98.0,2.0,10.0,16.0,33.0,57.0,87.0,128.0,457.0,289.0,790.0,1177.0
5,201004.0,112.0,3.0,9.0,25.0,25.0,52.0,97.0,130.0,468.0,276.0,809.0,1197.0
6,201005.0,102.0,3.0,7.0,28.0,41.0,67.0,137.0,152.0,462.0,268.0,897.0,1267.0


In [172]:
df_drs = df[['DRs', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unrated.2', 'Unnamed: 40', 'Unnamed: 41']]

df_drs.columns = df_drs.loc[0,'DRs':'Unnamed: 41']
df_drs = df_drs.drop([0,1])
df_drs = df_drs.rename(columns={df_drs.columns[0]: 'data'})

df_drs[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated >500',          'TOTAL']] = df_drs[['<=500_1',   '>500_Rated_1',
         '>500_Rated_2',   '>500_Rated_3',   '>500_Rated_4',   '>500_Rated_5',
         '>500_Rated_6',   '>500_Rated_7',   '>500_Rated_8', '>500_Unrated_1',
          'Rated >500',          'TOTAL']].astype(float)

df_drs.columns.name = 'DRs'
df_drs.head().columns

Index(['data', '<=500_1', '>500_Rated_1', '>500_Rated_2', '>500_Rated_3',
       '>500_Rated_4', '>500_Rated_5', '>500_Rated_6', '>500_Rated_7',
       '>500_Rated_8', '>500_Unrated_1', 'Rated >500', 'TOTAL'],
      dtype='object', name='DRs')

In [173]:
df_drs = df_drs.rename(columns={'Rated >500': 'Rated >500'})
df_defaults = df_defaults.rename(columns={'Rated>500': 'Rated >500'})
df_bons = df_bons.rename(columns={'Rated > 500': 'Rated >500'})

In [174]:
df_drs.columns

Index(['data', '<=500_1', '>500_Rated_1', '>500_Rated_2', '>500_Rated_3',
       '>500_Rated_4', '>500_Rated_5', '>500_Rated_6', '>500_Rated_7',
       '>500_Rated_8', '>500_Unrated_1', 'Rated >500', 'TOTAL'],
      dtype='object', name='DRs')

In [175]:
df_drs.to_csv('drs.csv', index=False)
df_bons.to_csv('bons.csv', index=False)
df_defaults.to_csv('defaults.csv', index=False)

In [176]:
df_defaults.columns

Index(['data', '<=500_1', '>500_Rated_1', '>500_Rated_2', '>500_Rated_3',
       '>500_Rated_4', '>500_Rated_5', '>500_Rated_6', '>500_Rated_7',
       '>500_Rated_8', '>500_Unrated_1', 'Rated >500', 'TOTAL'],
      dtype='object', name='DEFAULTS')